## 4 Model IO

https://python.langchain.com/docs/modules/model_io/

https://python.langchain.com/docs/modules/data_connection/

### 4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
    "country" : "아이슬란드"
})

### 4.2 FewShotChatMessagePromptTemplate 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 예시 문장과 실제 예시를 보여줌   
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# system : 전달 인자, example_prompt : 예시 문장과 실제 예시 프롬프트, human : 물어볼 메시지
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

print(f"3 : {final_prompt}")

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

### 4.3 LengthBasedExampleSelector

example의 수를 제한해서 비용을 줄일 수 있는 방법

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")

example_selector = RandomExampleSelector(
    examples = examples,
)
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="브라질")

### 4.4 Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

prompt.format(country="xxx")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

### 4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)


chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pizza")

### 4.6 Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import SQLiteCache

# set_llm_cache(SQLiteCache("cache.db"))
# set_debug(True)

chat = ChatOpenAI(
    # model_name = "gpt-4-0125-preview",
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    max_tokens= 1000,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for toast")
    b = chat.predict("What is the recipe for egg sandwich")
    print(a, "\n")
    print(b, "\n")
    print(usage)